# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [2]:
import re
from collections import Counter
from tqdm import tqdm
import random

In [3]:
# Define special tokens
SPECIAL_TOKENS = {'<NUM>', '<UNK>'}

The dataset for a training part is a hybrid of two datasets. The first one is a big.txt file from Norvig's solution, and the second one is a bigrams.txt file from the ngrams dataset. The ones can be downloaded manually from the following resources:

- big.txt - https://norvig.com/big.txt
- bigrams.txt - provided in the assignment on moodle

In [4]:
# Load ngrams from bigrams.txt which is structured as count\tword1\tword2
N_GRAMS = Counter()
with open('bigrams.txt', 'r', encoding='utf-8') as f:
    for line in f:
        count, word1, word2 = line.split()
        N_GRAMS[(word1, word2)] = int(count)

print(len(N_GRAMS))
N_GRAMS.most_common(7)

1020385


[(('of', 'the'), 2586813),
 (('in', 'the'), 2043262),
 (('to', 'the'), 1055301),
 (('on', 'the'), 920079),
 (('and', 'the'), 737714),
 (('to', 'be'), 657504),
 (('at', 'the'), 617976)]

In [5]:
# Calculate probabilities for each n-gram
total_n_grams = sum(N_GRAMS.values())
n_gram_probs = {n_gram: count / total_n_grams for n_gram, count in N_GRAMS.items()}

Here we can drop the numbers and punctuation from the text because the task of spellchecking is to correct the words, not the numbers or punctuation. We can also lowercase the text to make it easier to work with.

In [6]:
# Load the big.txt dataset and preprocess it
text = open('big.txt').read().lower()
text = re.sub(r'\d+', '<NUM>', text)
text = re.sub(r'[^\w\s]', '', text)
text = re.findall(r'\w+', text)
WORDS = Counter(text)

print(len(WORDS))
WORDS.most_common(7)

37429


[('the', 79178),
 ('of', 39995),
 ('and', 38092),
 ('to', 28610),
 ('in', 21754),
 ('a', 20843),
 ('he', 12182)]

Now let us describe the main algorithm. So the idea is to mix the spellchecked word predictions of Norvig's solution with the predictions of the n-gram model. The algorithm tries to increase the accuracy and add the context to such Norvig's solution. The algorithm is as follows:

In [7]:
def predict_word(word, n):
    """
    Predict the first or the second word in bigram, given the other word
    :param word: str - the word to predict from
    :param n: int - the position of the word in the bigram,
    0 - if predict first word given the second,
    1 - if predict the second word given the first
    :return: the tuple of the predicted words and its probabilities
    """
    next_word_probs = {}

    # Iterate over all n-grams and get the probabilities of the next word
    for n_gram, prob in n_gram_probs.items():

        # Add the probability of the predicted words
        if n_gram[n-1] == word:
            next_word_probs[n_gram[n]] = prob

    return next_word_probs


def P(word, N=sum(WORDS.values())):
    """
    Probability of `word`.
    :param word: str - the word to get the probability of
    :param N: int - the total number of words in the dataset
    :return: float - the probability of the word
    """
    return WORDS[word] / N


def check_correctness(word):
    """
    Check if the word is correct or not according to the Norvig's solution
    :param word: str - the word to check
    :return: bool, tuple - the flag if the word is correct and the candidates for the word
    """

    # If word is a special token, then it is correct
    if word in SPECIAL_TOKENS:
        return True, {}

    # Get the candidates
    candidates = get_candidates(word)

    # if it is only one candidate and it is the same as the word, then it is correct
    if len(candidates) == 1 and word in candidates:
        return True, candidates

    # if the word is in the dictionary, then it is incorrect
    else:
        return False, candidates


def correction(word):
    """
    Get the most probable correction of the word
    :param word: str - the word to correct
    :return: str - the corrected word
    """
    return max(get_candidates(word), key=P)


def get_candidates(word):
    """
    Get the candidates for the word
    :param word: str - the word to get the candidates for
    :return: list - the list of candidates
    """
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])


def known(words):
    """
    Get the words that are in the dictionary
    :param words: list - the list of words to check
    :return: set - the set of words that are in the dictionary
    """
    return set(w for w in words if w in WORDS)


def edits1(word):
    """
    Get all edits that are one edit away from `word`.
    :param word: str - the word to get the edits for
    :return: set - the set of edits
    """
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    """
    Get all edits that are two edits away from `word`.
    :param word: str - the word to get the edits for
    :return: set - the set of edits
    """
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


def fix_sentence(sentence):
    """
    The main algorithm to fix the sentence
    :param sentence: str - the sentence to fix
    :return: str - the fixed sentence
    """

    # Preprocess the sentence as were done before
    sentence = sentence.lower()
    sentence = re.sub(r'[^\w\s]', '', sentence)
    sentence = re.sub(r'\d+', '<NUM>', sentence)

    # Tokenize the sentence
    words = sentence.split()

    # Iterate over all words
    for i, word in enumerate(words):
        check, candidates = check_correctness(word)

        # If the word is incorrect, then fix it
        if not check:

            # Get the Norvig's probabilities and couple them with candidates
            norv_probs = {}
            for c in candidates:
                norv_probs[c] = P(c)
            norv_probs = dict(sorted(norv_probs.items(), key=lambda item: item[1], reverse=True))

            # Get two bigrams before and after the word if possible and make predictions
            ngram_probs = {}
            if i >= 1 and i <= len(words) - 2:
                ngram_probs = {**predict_word(words[i-1], 1), **predict_word(words[i+1], 0)}

            elif i == 0:
                ngram_probs = predict_word(words[i+1], 0)

            elif i == len(words) - 1:
                ngram_probs = predict_word(words[i-1], 1)



            # Normalize the probabilities
            total_norv = sum(norv_probs.values())
            total_ngram = sum(ngram_probs.values())
            norv_probs = {k: v / total_norv for k, v in norv_probs.items()}
            ngram_probs = {k: v / total_ngram for k, v in ngram_probs.items()}

            # Сombine the prediction words with the candidates and if the key is the same, then take an average of this two
            prediction_set = norv_probs
            for p in ngram_probs:
                if p in norv_probs:
                    prediction_set[p] = (prediction_set[p] + ngram_probs[p]) / 2
                else:
                    prediction_set[p] = ngram_probs[p]

            # Get the most probable word and fix the sentence
            word_pred = max(prediction_set, key=prediction_set.get)
            words[i] = word_pred

    return " ".join(words)

In [8]:
fix_sentence("Becouse, i likke the 1 is honoroded oon the theees")

'because i like the <NUM> is honored on the theres'

## Justifying the decisions

The Norvig's solution is a good spellchecker, but it does not take into account the context of the words. The n-gram model, on the other hand, is a good model to predict the next word given the previous one. So the idea is to combine these two models to increase the accuracy of the spellchecker.

To simplify the algorithm, the Norvig's solution is used to detect words need to be corrected. So if the one did not offer the candidates at all or offer single one similar to check word, then the word treated as 'incorrect'. The approach is used because the methodology of the one is approved and well-known. Moreover, in case we want to improve the accuracy of Norvig's solution, such tagging ('correct', 'incorrect') is obligatory for the n-gram to further deal only with context dependent cases.

The data used for the training part is a hybrid of two datasets. The first one is a big.txt file from Norvig's solution. I have chosen this dataset because it is a very good balanced corpus of English text. During the experiments with much bigger datasets ([wikipedia](https://www.kaggle.com/datasets/jkkphys/english-wikipedia-articles-20170820-sqlite), [english books](https://www.kaggle.com/datasets/raynardj/classic-english-literature-corpus), etc.) it was found that the word dictionary seems to be hard to filter and containing a lot of words with 'spell' mistakes.

As for the N-gram dataset, I have chosen the bigrams.txt file from the ngrams dataset. The experiments show that the bigrams are the most suitable for the task of spellchecking because. Fivegrams and trigrams results in a high number of variations but with low possibility of being fitted to the context. In such cases, the n-gram model simply did not give any predictions, since that bigrams are used.

Another important thing is the probability aggregation. In my model, the N-gram and Norvig's solutions are equally weighted. So to get the final prediction, the probabilities of the two models are normalized and averaged if both predicts the same word with different probability. Other words added to the total prediction set by union of the two models predictions.

## Evaluate on a test set

For the evaluation part, it is used the [wikipedia-sentences](https://www.kaggle.com/datasets/mikeortman/wikipedia-sentences) dataset from kaggle. To make the evaluation more efficient, only the first 1000 sentences are used. The sentences are preprocessed by removing the punctuation, numbers, and lowercasing.

In [9]:
data = open('wikisent2.txt', 'r', encoding='utf-8').readlines()

# Shuffle the data
random.seed(42)
random.shuffle(data)

# Get only first 1000 sentences
data = data[:1000]

# Lowercase, remove punctuation, /n, and replace numbers with <NUM>
data = [re.sub(r'[^\w\s]', '', line.lower()) for line in data]
data = [re.sub(r'\d+', ' <NUM> ', line) for line in data]
data = [line.replace('\n', '') for line in data]
data = [re.sub(' +', ' ', line) for line in data]

print(len(data))
data[:4]

1000


['imanol sarriegi isasa born <NUM> april <NUM> is a spanish footballer who plays as a central midfielder',
 'indeed the greater part of this chisian daniel cannot be said to deserve the name of a translation at all',
 'after moving to los angeles to become an actor rambo started working in the real estate business',
 'the rolladenschneider ls <NUM> is a standard class single seat glider manufactured by rolladenschneider flugzeugbau gmbh between <NUM> and <NUM> ']

To add the noise to the data I will simply modify each word in the sentence with a certain probability. So the word can be modified with 3 different ways:
- Change a random letter
- Add a random letter
- Remove a random letter

In [11]:
word_mistake_probability = 0.3          # Probability of a single word to be incorrect
letter_change_prob = 0.4                # Probability of a letter to be randomly changed
additional_letter_probability = 0.2     # Probability of a letter to be randomly added
missing_letter_probability = 0.2        # Probability of a letter to be randomly removed


def make_mistake(word):
    """
    The function to make a mistake in the word
    :param word: str - the word to make a mistake in
    :return: str - the word with a mistakes (or not)
    """

    # If the word is a special token, then do not make a mistake
    if random.random() < word_mistake_probability and word not in SPECIAL_TOKENS:
        if random.random() < letter_change_prob:
            # Change a random letter
            word = list(word)
            word[random.randint(0, len(word) - 1)] = random.choice('abcdefghijklmnopqrstuvwxyz')
            word = "".join(word)

        if random.random() < additional_letter_probability:
            # Add a random letter
            word = list(word)
            word.insert(random.randint(0, len(word)), random.choice('abcdefghijklmnopqrstuvwxyz'))
            word = "".join(word)

        if random.random() < missing_letter_probability:
            # Remove a random letter
            word = list(word)
            word.pop(random.randint(0, len(word) - 1))
            word = "".join(word)

    return word

# Make mistakes in the dataset
data_mistakes = []
for line in tqdm(data):
    words = line.split()
    words = [make_mistake(word) for word in words]
    data_mistakes.append(" ".join(words))

data_mistakes[:4]

100%|██████████| 1000/1000 [00:00<00:00, 54074.70it/s]


['imanol sarriegi isasa born <NUM> april <NUM> is m spanish jotballer who plays as a central midfielder',
 'indeed the greater part of this chisian daniel cannot be said to deserve the nae of a translation at all',
 'after moxing to los anheles tm become an actor rambo started working in the real estate business',
 'the rolladenschneider l <NUM> j a standard class single reat glider manufactured by rolladenschneider flugzeugbau gmbh between <NUM> and <NUM>']

In [12]:
# Fix the sentences via my solution
fix_sentences = []
for line in tqdm(data_mistakes):
    fix_sentences.append(fix_sentence(line))

100%|██████████| 1000/1000 [12:32<00:00,  1.33it/s]


The evaluation metric is the accuracy. To make it more precise the one is done via words (not sentences). So the function to get the similarity of two sentences is implemented.

In [13]:
def get_similarity(sentence1, sentence2):
    """
    Get the similarity of two sentences via accuracy of guessing words.
    :param sentence1: str - the first sentence
    :param sentence2: str - the second sentence
    :return: float - the accuracy of sentence guessing
    """
    words1 = sentence1.split()
    words2 = sentence2.split()

    correct = 0
    total = 0

    for i in range(len(words1)):
        if words1[i] == words2[i]:
            correct += 1
        total += 1

    return correct / total

In [14]:
avg_acc = 0
for i, sentence in enumerate(data_mistakes):
    avg_acc += get_similarity(sentence, fix_sentences[i])

avg_acc = avg_acc / len(data_mistakes)
avg_acc

0.7636947190242973

In [15]:
# Fix and evaluate the sentences via my Norvig's solution
avg_acc = 0
for i, sentence in enumerate(tqdm(data_mistakes)):
    avg_acc += get_similarity(sentence, " ".join([correction(word) for word in data_mistakes[i].split()]))

avg_acc = avg_acc / len(data_mistakes)
avg_acc

100%|██████████| 1000/1000 [03:04<00:00,  5.41it/s]

0.7334544590272988


The difference between the accuracy of my solution and Norvig's one is not so significant but seems to be stable. The accuracy of my solution is a bit greater, probably because it captures the context based dependencies or simply because it increases the possible prediction candidates for the word.

However, important to note that the time efficiency of my algorithm is much worse than Norvig's one. So the one need to be optimized.

Finally, mistake probabilities for the evaluation dataset are chosen relatively high. This is done to make the evaluation more challenging and probably create the situations where the context dependent cases occure more often. In case of lower mistake probabilities, the accuracy of the Norvig's solution and my solution will be much closer.